# Estrategias Evolutivas

### (1 + 1) - EE

In [38]:
import numpy as np
import math
import copy
import random

In [39]:
def f(x):
    sumatoria1 = 0
    sumatoria2 = 0
    for x_ in x:
        sumatoria1 += x_**2
        sumatoria2 += math.cos(x_ * math.pi * 2)
    p1 = -20*(math.e**(-0.2*(math.sqrt(((1/len(x)*sumatoria1))))))
    p2 = -math.e**(((1/len(x))*sumatoria2))

    return p1 + p2 + 20 + math.e

In [40]:
def adecuacion(x):
    return -f(x)

In [41]:
def generar_solucion_inicial(n, sigma):
    x_i = [np.random.uniform(-30, 30) for _ in range(n)]
    return [x_i, sigma]

In [42]:
def mutacion(x, sigma):
    x_nueva = [i + (sigma * np.random.normal(0, 1)) for i in x[0]]
    return [x_nueva, sigma]

In [43]:
def evolucion_uno_uno(sigma, cons, G, num_vars):
    solucion = generar_solucion_inicial(num_vars, sigma)
    mejoras = 0
    i = 0
    while i < G:
#         print(i)
#         print(solucion, adecuacion(solucion[0]))
        nueva_solucion = mutacion(solucion, sigma)
        if adecuacion(nueva_solucion[0]) > adecuacion(solucion[0]):
            solucion = copy.deepcopy(nueva_solucion)
            mejoras += 1
        i += 1

        #Definí checar la relación de las mejoras cada 10 iteraciones
        if i % 5 == 0: 
#             print('checando mejoras  ', mejoras)
            p_s = mejoras/5
            if p_s > 1/5:
                sigma = sigma/cons
            elif p_s < 1/5:
                sigma = sigma * cons
            mejoras = 0
#             print('nueva sigma   ', sigma)

    return solucion

In [50]:
solucion = evolucion_uno_uno(0.5, 0.817, 50, 3)
print('solucion', solucion[0])
print('adecuacion', solucion[1])

solucion [4.96147508769996, 21.009713474563334, 18.976778627492592]
adecuacion 0.12148548730138936


### (u, $\lambda$) - EE

In [45]:
def generar_poblacion_inicial(u, n):
    poblacion = []
    for _ in range(u):
        x_i = [np.random.uniform(-30, 30) for _ in range(n)]
        s_i = np.random.uniform(0, 1) 
        p = adecuacion(x_i)
        # s_i = sigma 
        poblacion.append([x_i, s_i, p])
    return poblacion

In [46]:
def elegir_padres(poblacion):
    return random.sample(poblacion, 1)[0], random.sample(poblacion, 1)[0]

def cruza_u_lambda(padres):
    padre1 = padres[0]
    padre2 = padres[1]
    hijo = []
    for i in range(len(padre1[0])):
        a = np.random.uniform(0,1)
        if a < 0.5:
            hijo.append(padre1[0][i])
        else:
            hijo.append(padre2[0][i])

    cruza = [hijo]
    a = np.random.uniform(0,1)
    if a < 0.5:
        cruza.append(padre1[1])
    else:
        cruza.append(padre2[1])

    p = adecuacion(cruza[0])
    cruza.append(p)

    return cruza


def mutacion_u_lambda(hijo, epsilon, tau):
    variables_mutadas = []
    sigma_n = hijo[1] * (math.e**((tau*np.random.normal(0,1))))
    if sigma_n < epsilon:
        sigma_n = epsilon
    for x in hijo[0]:
        x_i = x * (np.random.normal(0,1)*sigma_n)
        variables_mutadas.append(x_i)

    mutado = [variables_mutadas, sigma_n]
    mutado.append(adecuacion(variables_mutadas))
    return mutado

def generar_hijo(padres, epsilon, tau):
    cruza_ = cruza_u_lambda(padres)
    mutacion_ = mutacion_u_lambda(cruza_, epsilon, tau)
    return mutacion_

In [47]:
def evolucion_u_lambda(G, tau, epsilon, u, lambda_, n):
    poblacion = generar_poblacion_inicial(u, n)
    generaciones = 0
    while generaciones < G:
        cont = 0
        hijos = []
        while cont < lambda_:
            padres = elegir_padres(poblacion)
            hijo = generar_hijo(padres, epsilon, tau)
            hijos.append(hijo)
            cont += 1

        poblacion = sorted(hijos, key= lambda x: x[-1], reverse=True)
        poblacion = poblacion[:u]

        generaciones += 1

    return poblacion[-1]

In [51]:
solucion = evolucion_u_lambda(50, 1/(math.sqrt(3)), 0.01, 30, 50, 3)
print('solucion', solucion[0])
print('adecuacion', solucion[2])

solucion [9.38801586255652e-22, 7.116041828769058e-19, -3.6548731082723396e-19]
adecuacion -4.440892098500626e-16


### (u + $\lambda$, k) - EE

En este caso la estrategía propuesta por mi fue la implementación de una conocida estrategia en la que se considera la edad "k" de los individuos. Para esta estrategia lo que pasa es que la población se compone de los mejores u entre los padres y los hijos (u + $\lambda$) pero después de cada generación se evalua la edad de cada individuo creado (que incrementa en uno después de cada generación) y si el individuo tiene una mayor edad que la establecida en los parametros, se elimina y se genera una solución nueva de manera aleatoria para completar la población. Un punto importante es que esto no pasa si el individuo es el mejor de la población.

Esta estrategia ayuda a que se exploren más posibilidades pues un individuo puede quedarse estancado y quedarse en la población por mucho tiempo por ser muy buena solución pero si no es la mejor, entonces realmente no nos interesa y es una solución estancada que le roba espacio a otras soluciones que podrían ser mejores, por lo cual lo que queremos con esta estrategia es quitar esos individuos para dar lugar a otros individuos que podrían evolucionar a una mejor solución.

In [74]:
import lambda_k_u_ackley as u_k_lambda

In [76]:
respuesta = u_k_lambda.evolucion(1000, 1/(math.sqrt(3)), 0.01, 30, 50, 3, 10)
print(respuesta.variables, respuesta.adecuacion, respuesta.edad)

[1.1579064791576834e-16, 1.804960378736281e-17, 1.0389666770389798e-16] -4.440892098500626e-16 992


## Permitir al usuario realizar M ejecuciones

### 5 variables

In [77]:
resultados = []
M = 50
for _ in range(M):
    respuesta = u_k_lambda.evolucion(1000, 1/(math.sqrt(5)), 0.01, 50, 70, 5, 10)
    resultados.append(respuesta)

In [78]:
resultados = sorted(resultados, key= lambda x: x.adecuacion, reverse=True)

- mejor solución 

In [90]:
print('variables ->', resultados[49].variables)
print('f(x) = ', resultados[49].adecuacion)

variables -> [-2.4325995873492205e-16, 3.506345315134679e-22, -2.460094331169217e-18, 3.8263851411953026e-20, -2.1682525532209714e-16]
f(x) =  -4.440892098500626e-16


- peor solucion

In [91]:
print('variables ->', resultados[0].variables)
print('f(x) = ', resultados[0].adecuacion)

variables -> [4.593809952950315e-18, 2.330836933451049e-17, -1.1088787031728511e-21, -1.1758156818580967e-17, -1.8885210425451648e-16]
f(x) =  -4.440892098500626e-16


- mediana

In [93]:
print('variables ->', resultados[M//2].variables)
print('f(x) = ', resultados[M//2].adecuacion)

variables -> [1.778773794556364e-18, 3.8566648584040125e-18, -2.9989259312333017e-19, -2.2618688155961693e-20, 3.805174288071079e-16]
f(x) =  -4.440892098500626e-16


- promedio y desviación estandar de f(x)

In [96]:
suma = []
for r in resultados:
    suma.append(r.adecuacion)

In [98]:
from statistics import stdev

In [100]:
prom = sum(suma)/M
st_dev = stdev(suma)
print('promedio -> ', prom)
print('desviación estándar -> ', st_dev)

promedio ->  -4.440892098500626e-16
desviación estándar ->  0.0


### 20 variables

In [101]:
resultados = []
M = 50
for _ in range(M):
    respuesta = u_k_lambda.evolucion(1000, 1/(math.sqrt(20)), 0.01, 50, 70, 20, 10)
    resultados.append(respuesta)

In [102]:
resultados = sorted(resultados, key= lambda x: x.adecuacion, reverse=True)

- mejor solución

In [103]:
print('variables ->', resultados[49].variables)
print('f(x) = ', resultados[49].adecuacion)

variables -> [1.4376168481451512e-19, 1.4673269872106488e-16, 6.682033887484706e-16, -7.271299710905303e-20, -3.658716073367745e-20, -7.271647734450848e-18, -1.2557951359223216e-20, -2.711900084520165e-18, -1.3113664929910108e-20, 4.09520142224782e-19, -1.4080836510025434e-17, -8.330939372456354e-17, -5.4753652016516555e-18, -8.539488659595363e-21, 6.902576141493296e-17, 1.2143192812549266e-19, 1.4014974604620454e-21, -7.162340010631991e-18, 4.0409102215735297e-19, 3.792250739350536e-18]
f(x) =  -4.440892098500626e-16


- peor solución

In [104]:
print('variables ->', resultados[0].variables)
print('f(x) = ', resultados[0].adecuacion)

variables -> [-2.5328493501781514e-19, 6.293528475687554e-16, 4.9034536586707394e-17, -4.642988789756193e-18, 7.402517796148102e-18, 5.902221930277619e-17, -3.495966035987909e-18, 1.9020047103715635e-18, -6.486032920745322e-18, -1.1731005347957365e-17, -1.3380841001206044e-16, 3.698119253739083e-16, -1.1525229417195323e-18, 6.747865943568043e-19, 1.10846069313103e-16, -6.914668431874608e-19, -8.77398387986246e-17, -1.6674372146700703e-17, -1.4149539140107463e-16, 8.421314819107877e-17]
f(x) =  -4.440892098500626e-16


- mediana

In [105]:
print('variables ->', resultados[M//2].variables)
print('f(x) = ', resultados[M//2].adecuacion)

variables -> [2.749105998656843e-18, -8.905469024275474e-17, 4.934370675751175e-19, -6.123445199930384e-18, -2.6379915678647936e-19, 2.2315509094698616e-19, 2.837313470236943e-16, 4.418824536376109e-19, 8.105455957284205e-19, 7.549005887902665e-20, -7.29266953469106e-18, -1.5087428037505673e-19, 3.895973243817834e-18, -1.0532584548277125e-19, -7.717729490747697e-19, -5.2391706836924464e-17, 1.3538067299702302e-19, -9.076439082702798e-20, 2.848957256025694e-16, 1.477136781204566e-17]
f(x) =  -4.440892098500626e-16


- promedio y desviación estándar de f(x)

In [106]:
prom = sum(suma)/M
st_dev = stdev(suma)
print('promedio -> ', prom)
print('desviación estándar -> ', st_dev)

promedio ->  -4.440892098500626e-16
desviación estándar ->  0.0


### 10 variables

In [107]:
resultados = []
M = 50
for _ in range(M):
    respuesta = u_k_lambda.evolucion(1000, 1/(math.sqrt(10)), 0.01, 50, 70, 10, 10)
    resultados.append(respuesta)

In [108]:
resultados = sorted(resultados, key= lambda x: x.adecuacion, reverse=True)

- mejor solución

In [109]:
print('variables ->', resultados[49].variables)
print('f(x) = ', resultados[49].adecuacion)

variables -> [6.6591824330424286e-18, 1.3427319953326538e-17, -1.258726314588867e-18, -3.5533961714136317e-16, 1.5079982226884037e-16, 1.439679491211589e-17, 3.143449571851503e-18, 4.2278697870159606e-17, 9.901332436473076e-18, 4.402702565586702e-19]
f(x) =  -4.440892098500626e-16


- peor solución 

In [110]:
print('variables ->', resultados[0].variables)
print('f(x) = ', resultados[0].adecuacion)

variables -> [-4.4610799540992244e-18, 3.1231289832629523e-16, -5.0588436188835994e-20, -8.62862684635649e-17, -2.4186985875250683e-17, -3.6313415091125996e-16, 2.9874091859653585e-19, 5.419822046256503e-20, -1.4144896315279087e-18, -6.340368032476911e-17]
f(x) =  -4.440892098500626e-16


- mediana

In [111]:
print('variables ->', resultados[M//2].variables)
print('f(x) = ', resultados[M//2].adecuacion)

variables -> [7.228765752938913e-18, -4.128350345219051e-20, -1.0461681708791677e-21, 3.232146601996725e-20, 1.5230128494043415e-20, 5.11202022317665e-16, 1.471941401364959e-16, 1.1732202343923906e-17, 5.39437150502233e-19, 9.064044011777468e-23]
f(x) =  -4.440892098500626e-16


- promedio y desviación estándar de f(x)

In [112]:
prom = sum(suma)/M
st_dev = stdev(suma)
print('promedio -> ', prom)
print('desviación estándar -> ', st_dev)

promedio ->  -4.440892098500626e-16
desviación estándar ->  0.0


## Comparación de las tres versiones

### 2 variables

- (1 + 1) - EE

In [114]:
solucion = evolucion_uno_uno(0.5, 0.817, 100, 2)
print(solucion[0])
print('f(x) = ', solucion[1])

[29.006098900822217, 14.0000892496775]
f(x) =  0.054126834151748376


- (u, $\lambda$) - EE

In [115]:
solucion = evolucion_u_lambda(100, 1/(math.sqrt(2)), 0.01, 50, 70, 2)
print(solucion[0])
print('f(x) = ', solucion[2])

[4.726597031086321e-18, -2.7669030566891397e-17]
f(x) =  -4.440892098500626e-16


- (u + $\lambda$, k) - EE

In [116]:
respuesta = u_k_lambda.evolucion(100, 1/(math.sqrt(2)), 0.01, 50, 70, 2, 10)
print(respuesta.variables)
print('f(x) = ', respuesta.adecuacion)

[-2.452987042085751e-16, -3.2088293752787446e-17]
f(x) =  -4.440892098500626e-16


### 5 variables

- (1 + 1) - EE

In [117]:
solucion = evolucion_uno_uno(0.5, 0.817, 100, 5)
print(solucion[0])
print('f(x) = ', solucion[1])

[-12.96867586100382, -4.214337210457798, 3.9263811476756607, 29.964562006530304, -11.967959017611328]
f(x) =  0.12148548730138932


- (u, $\lambda$) - EE

In [118]:
solucion = evolucion_u_lambda(100, 1/(math.sqrt(5)), 0.01, 50, 70, 5)
print(solucion[0])
print('f(x) = ', solucion[2])

[-3.992013732927e-18, 1.1286921224849475e-17, -6.792726176074808e-18, 4.959245361588046e-16, 1.094358306464585e-16]
f(x) =  -4.440892098500626e-16


- (u + $\lambda$, k) - EE

In [120]:
respuesta = u_k_lambda.evolucion(100, 1/(math.sqrt(5)), 0.01, 50, 70, 5, 10)
print(respuesta.variables)
print('f(x) = ', respuesta.adecuacion)

[-3.2068485093845375e-17, 5.20905120806007e-20, 5.3098438528604406e-17, -5.075762797428315e-18, 9.97297291639815e-18]
f(x) =  -4.440892098500626e-16


### 7 variables

- (1 + 1) - EE

In [121]:
solucion = evolucion_uno_uno(0.5, 0.817, 100, 7)
print(solucion[0])
print('f(x) = ', solucion[1])

[14.916734604032573, 10.97731817087725, 5.932301552701655, -12.013124073561103, 1.9814600819209165, -7.045842227981956, 6.919064156858902]
f(x) =  0.04422162350197842


- (u, $\lambda$) - EE

In [122]:
solucion = evolucion_u_lambda(100, 1/(math.sqrt(7)), 0.01, 50, 70, 7)
print(solucion[0])
print('f(x) = ', solucion[2])

[5.887545284225804e-19, -4.1700453670095894e-23, 3.33283517506513e-20, -2.0797286863534818e-19, -5.189656811404056e-22, -2.3213795748919115e-20, 1.396164968950573e-22]
f(x) =  -4.440892098500626e-16


- (u + $\lambda$, k) - EE

In [123]:
respuesta = u_k_lambda.evolucion(100, 1/(math.sqrt(7)), 0.01, 50, 70, 7, 10)
print(respuesta.variables)
print('f(x) = ', respuesta.adecuacion)

[-2.5566861813279e-18, -2.893616992354594e-16, -3.6967220702125685e-18, -8.383230701579134e-17, -3.115540555495894e-17, 5.315366204195172e-19, -7.234313289881635e-17]
f(x) =  -4.440892098500626e-16


### 10 variables

- (1 + 1) - EE

In [124]:
solucion = evolucion_uno_uno(0.5, 0.817, 100, 10)
print(solucion[0])
print('f(x) = ', solucion[1])

[1.983859760742172, 2.0308578618899187, 7.991705799667852, -16.033183167478793, -24.088335138460046, 1.8986024536390094, 12.058124380427628, 11.835712833438693, -2.085178800793914, -9.003326030354529]
f(x) =  0.0442216235019784


- (u, $\lambda$) - EE

In [125]:
solucion = evolucion_u_lambda(100, 1/(math.sqrt(10)), 0.01, 50, 70, 10)
print(solucion[0])
print('f(x) = ', solucion[2])

[-1.3642714942236896e-19, -5.825669416446552e-18, -4.1174551521641414e-18, 1.5256044193005944e-18, 8.832259488759074e-20, 1.0746275688291284e-17, 2.8380809436047048e-18, -1.519824685221115e-22, 6.594857575895366e-23, -3.382881519863325e-20]
f(x) =  -4.440892098500626e-16


- (u + $\lambda$, k) - EE

In [126]:
respuesta = u_k_lambda.evolucion(100, 1/(math.sqrt(10)), 0.01, 50, 70, 10, 10)
print(respuesta.variables)
print('f(x) = ', respuesta.adecuacion)

[-7.638172407337447e-17, 4.491667648529542e-17, -6.521558643293475e-22, -9.618331861327014e-20, 3.169436503582194e-18, -4.8256350869983885e-18, -2.2774096914613437e-18, 6.531020682112341e-26, -1.1868355183931678e-18, -1.8524412589347444e-17]
f(x) =  -4.440892098500626e-16
